In [ ]:
import pandas as pd

In [ ]:
!git clone https://github.com/BNIA/tidyaddr-js.git

In [ ]:
! npm install tidyaddr-js/

In [ ]:
cd tidyaddr-js

In [ ]:
pd.read_excel(open('EnochPratt_FY2020_VS19.xlsx', 'rb'), sheet_name='AllCardTypes2020').to_csv('lib.csv')

In [ ]:
ls

In [ ]:
!node tidyaddr.js clean-csv lib.csv tidyaddrd.csv

In [ ]:
df = pd.read_csv("tidyaddrd.csv");

In [ ]:
df.head(1)

In [ ]:
df.to_excel('output1.xlsx', engine='xlsxwriter') ;
df.head(1)

In [ ]:
%%capture
! pip install geopy
! pip install geopandas
! pip install geoplot
! pip install dataplay

In [ ]:
ls

In [ ]:
%%capture 
# @title Run: Import Modules

# These imports will handle everything
import os
import sys
import csv
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim
import folium
from folium import plugins

from dataplay.merge import mergeDatasets 

import dexplot as dxp

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:

address_df = df.copy()
addrCol = 'address'

# An example of loading in an internal BNIA file
crs = {'init' :'epsg:2248'} 

In [ ]:
# More information vist: https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders

# In this example we retrieve and map a dataset with no lat/lng but containing an address

# In this example our data is stored in the 'STREET' attribute
geometry = []
geolocator = Nominatim(user_agent="my-application")

for index, row in address_df.iterrows():
  # We will try and return an address for each Street Name
  try: 
      # retrieve the geocoded information of our street address
      geol = geolocator.geocode(row[addrCol], timeout=None)

      # create a mappable coordinate point from the response object's lat/lang values.
      pnt = Point(geol.longitude, geol.latitude)
      
      # Append this value to the list of geometries
      geometry.append(pnt)
      
  except: 
      # If no street name was found decide what to do here.
      # df.loc[index]['geom'] = Point(0,0) # Alternate method
      geometry.append(Point(0,0))
      
# Finally, we stuff the geometry data we created back into the dataframe
address_df['geometry'] = geometry

In [ ]:
# save after every 100th geocode